In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Start a Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PCard Transactions").getOrCreate()

23/11/08 23:17:14 WARN Utils: Your hostname, Justins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.125 instead (on interface en0)
23/11/08 23:17:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/08 23:17:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/2/pcard_transactions.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("pcard_transactions.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+----------+-------------+--------------------+--------------------+------------------------+--------------------+------+--------------------+--------------------+--------------------+----------------------------+
|Year-Month|Agency Number|         Agency Name|Cardholder Last Name|Cardholder First Initial|         Description|Amount|              Vendor|    Transaction Date|         Posted Date|Merchant Category Code (MCC)|
+----------+-------------+--------------------+--------------------+------------------------+--------------------+------+--------------------+--------------------+--------------------+----------------------------+
|    201307|         1000|OKLAHOMA STATE UN...|               Mason|                       C|    GENERAL PURCHASE|   890|               NACAS|07/30/2013 12:00:...|07/31/2013 12:00:...|        CHARITABLE AND SO...|
|    201307|         1000|OKLAHOMA STATE UN...|               Mason|                       C|        ROOM CHARGES|368.96|      SHERATON HOTEL|07

In [5]:
# Get the data types of the columns. 
df.dtypes

[('Year-Month', 'string'),
 ('Agency Number', 'string'),
 ('Agency Name', 'string'),
 ('Cardholder Last Name', 'string'),
 ('Cardholder First Initial', 'string'),
 ('Description', 'string'),
 ('Amount', 'string'),
 ('Vendor', 'string'),
 ('Transaction Date', 'string'),
 ('Posted Date', 'string'),
 ('Merchant Category Code (MCC)', 'string')]

In [6]:
from pyspark.sql import types as T
# Create a new DataFrame with an "Amount" column as a float datatype. 
df2 = df.withColumn('Amount', df.Amount.cast(T.DoubleType()))
df2.dtypes

[('Year-Month', 'string'),
 ('Agency Number', 'string'),
 ('Agency Name', 'string'),
 ('Cardholder Last Name', 'string'),
 ('Cardholder First Initial', 'string'),
 ('Description', 'string'),
 ('Amount', 'double'),
 ('Vendor', 'string'),
 ('Transaction Date', 'string'),
 ('Posted Date', 'string'),
 ('Merchant Category Code (MCC)', 'string')]

In [7]:
from pyspark.sql import functions as F
# Group the data on the Agency Name and get the maximum of the "Amount."
df_agency = df2.groupBy('Agency number').agg(F.max('Amount'))
df_agency.show()

+-------------+-----------+
|Agency number|max(Amount)|
+-------------+-----------+
|         1000|   27967.38|
|         2200|      405.3|
|        34600|     4179.2|
|         2000|    2165.68|
|        30800|   12715.06|
|        12500|     1850.0|
|        19000|     1800.0|
|         6500|     4375.0|
|        27000|    58875.0|
|        21500|    3089.16|
|         2500|  183367.93|
|         9200|    7031.76|
|         4900|    15223.3|
|        18500|     6038.0|
|        24000|     7351.5|
|        32600|    3510.27|
|        13100|   96190.38|
|        31000|     2399.2|
|         4000|    72060.0|
|        22000|     5000.0|
+-------------+-----------+
only showing top 20 rows



In [8]:
# Group the data on the Agency Name and get the total of the "Amount."
df_agency = df2.groupBy('Agency number').agg(F.sum('Amount'))
df_agency.show()

+-------------+-------------------+
|Agency number|        sum(Amount)|
+-------------+-------------------+
|         1000|3.377883994000085E7|
|         2200| 1821.1800000000003|
|        34600|  79138.18999999999|
|         2000|           18057.08|
|        30800| 1117130.0200000012|
|        12500| 28480.519999999993|
|        19000|  40846.32000000001|
|         6500| 122181.51999999997|
|        27000| 167896.35999999996|
|        21500| 23023.449999999997|
|         2500| 3697283.2199999937|
|         9200|  63968.06000000001|
|         4900|  651519.2400000001|
|        18500| 208050.81000000008|
|        24000|  326154.9900000001|
|        32600| 10831.349999999999|
|        13100|1.398887190000005E7|
|        31000|  68339.49000000003|
|         4000|  2285718.489999994|
|        22000|  156613.7499999999|
+-------------+-------------------+
only showing top 20 rows



In [9]:
# Group the data on the Agency Name and get the total of the "Amount" using the `agg()` function.
df_agency = df2.groupBy('Agency number').agg(F.sum('Amount'))
df_agency.show()

+-------------+-------------------+
|Agency number|        sum(Amount)|
+-------------+-------------------+
|         1000|3.377883994000085E7|
|         2200| 1821.1800000000003|
|        34600|  79138.18999999999|
|         2000|           18057.08|
|        30800| 1117130.0200000012|
|        12500| 28480.519999999993|
|        19000|  40846.32000000001|
|         6500| 122181.51999999997|
|        27000| 167896.35999999996|
|        21500| 23023.449999999997|
|         2500| 3697283.2199999937|
|         9200|  63968.06000000001|
|         4900|  651519.2400000001|
|        18500| 208050.81000000008|
|        24000|  326154.9900000001|
|        32600| 10831.349999999999|
|        13100|1.398887190000005E7|
|        31000|  68339.49000000003|
|         4000|  2285718.489999994|
|        22000|  156613.7499999999|
+-------------+-------------------+
only showing top 20 rows



In [10]:
# Group the data on the Agency Name and get the average of the "Amount" using the agg() function.
df_agency = df2.groupBy('Agency number').agg(F.avg('Amount'))
df_agency.show()

+-------------+------------------+
|Agency number|       avg(Amount)|
+-------------+------------------+
|         1000| 291.2094481658765|
|         2200|182.11800000000002|
|        34600|378.65162679425833|
|         2000|126.27328671328672|
|        30800| 574.3599074550135|
|        12500| 212.5411940298507|
|        19000|179.93973568281942|
|         6500| 293.7055769230769|
|        27000| 739.6315418502201|
|        21500|215.17242990654202|
|         2500| 651.0447649234009|
|         9200|310.52456310679617|
|         4900|  435.798822742475|
|        18500|229.13084801762125|
|        24000|241.41746113989646|
|        32600|180.52249999999998|
|        13100| 626.6854179733022|
|        31000| 295.8419480519482|
|         4000| 417.8644405850081|
|        22000| 241.3154853620954|
+-------------+------------------+
only showing top 20 rows



In [11]:
# Group the data on the Agency Name and get the count of transactions of the "Amount" using the agg() function.
df_agency = df2.groupBy('Agency number').agg(F.count('Amount'))
df_agency.show()

+-------------+-------------+
|Agency number|count(Amount)|
+-------------+-------------+
|         1000|       115995|
|         2200|           10|
|        34600|          209|
|         2000|          143|
|        30800|         1945|
|        12500|          134|
|        19000|          227|
|         6500|          416|
|        27000|          227|
|        21500|          107|
|         2500|         5679|
|         9200|          206|
|         4900|         1495|
|        18500|          908|
|        24000|         1351|
|        32600|           60|
|        13100|        22322|
|        31000|          231|
|         4000|         5470|
|        22000|          649|
+-------------+-------------+
only showing top 20 rows



In [14]:
df2.columns

['Year-Month',
 'Agency Number',
 'Agency Name',
 'Cardholder Last Name',
 'Cardholder First Initial',
 'Description',
 'Amount',
 'Vendor',
 'Transaction Date',
 'Posted Date',
 'Merchant Category Code (MCC)']

In [15]:
# Group the data on the cardholder last name and get the maximum transaction per cardholder using the agg() function.
df_agency = df2.groupBy('Cardholder Last Name').agg(F.max('Amount'))
df_agency.show()

+--------------------+-----------+
|Cardholder Last Name|max(Amount)|
+--------------------+-----------+
|           Fairbanks|    3310.23|
|           Worcester|       6.97|
|              GILROY|    2476.95|
|            Callaham|     3775.0|
|             Creager|    1062.24|
|               Dunne|    3812.62|
|             Edmonds|     892.27|
|            Wooliver|     1833.2|
|               Gerlt|     3582.0|
|             Guthals|     2113.5|
|              Ownbey|    1335.09|
|              Porter|    3655.99|
|                Frie|    1341.65|
|                Deal|     518.52|
|            McDowell|     2000.0|
|               Scott|     4157.0|
|              Wilson|     5000.0|
|            Wilbourn|    1463.21|
|           Robertson|    7909.28|
|                Goad|     3900.0|
+--------------------+-----------+
only showing top 20 rows



In [16]:
# Get the average transaction per cardholder using the agg() function.
# Group the data on the cardholder last name and get the maximum transaction per cardholder using the agg() function.
df_agency = df2.groupBy('Cardholder Last Name').agg(F.avg('Amount'))
df_agency.show()

+--------------------+------------------+
|Cardholder Last Name|       avg(Amount)|
+--------------------+------------------+
|           Fairbanks|441.81742857142854|
|           Worcester| 5.734999999999999|
|              GILROY|246.38892473118278|
|            Callaham| 401.3933962264152|
|             Creager|122.80977272727272|
|               Dunne|503.36099999999993|
|             Edmonds|118.90054545454545|
|            Wooliver|427.08142857142855|
|               Gerlt| 246.3202816901408|
|             Guthals|179.90105263157895|
|              Ownbey| 452.5236363636364|
|              Porter|182.50486187845306|
|                Frie| 361.6395238095239|
|                Deal|123.70599999999997|
|            McDowell|383.48324324324324|
|               Scott|  314.994703196347|
|              Wilson|283.88818092428716|
|            Wilbourn| 171.2954166666666|
|           Robertson| 329.7328614457831|
|                Goad|299.32178160919534|
+--------------------+------------